In [1]:
import pandas as pd
from engine import run3

In [2]:
df = pd.read_csv(r'C:\Users\Administrator\WZZ\trader\data\TA_1d.txt',names=["date","time","symbol","open","high","low","close","volume"])
df["symbol"] = df["symbol"].str.split(".").str[1]
df['date'] = pd.to_datetime(df['date']+"-"+df['time'])
df = df.sort_values(by="date")
df["lead"] = (df["volume"] == df.groupby("date")["volume"].transform("max")).astype(int)

df["atr"] = (df.groupby("symbol")[["high", "low", "close"]]
                 .apply(lambda group: (group[["high", "close"]].assign(close=group["close"].shift()).max(axis=1) -
                                       group[["low", "close"]].assign(close=group["close"].shift()).min(axis=1))
                        .rolling(window=20).mean().shift()).reset_index(level=0, drop=True))

df['LE'] = df.groupby('symbol')['high'].transform(lambda x: x.rolling(window=8).max())
df['LE'] = df['LE']-0.1*df['atr']
df['SE'] = df.groupby('symbol')['low'].transform(lambda x: x.rolling(window=12).min())
df['SE'] = df['SE']+0.1*df['atr']
df["LX"] = df.groupby('symbol')['low'].transform(lambda x: x.rolling(window=5).min())
df["SX"] = df.groupby('symbol')['high'].transform(lambda x: x.rolling(window=7).max())
df["LES"] = True
df["SES"] = True
df["LXS"] = True
df["SXS"] = True

columns_to_shift = ["LE", "SE", "LX", "SX", "LES", "LXS", "SES", "SXS"]
df[columns_to_shift] = df.groupby("symbol")[columns_to_shift].transform(lambda x: x.shift())

In [3]:
run = run3.engine()
run.symbols = df['symbol'].unique()
df1 = run.backtest(df)

In [4]:
df1

,start,end,symbol,entry,exit,position,shares,pnl,PNL,ce,maxloss,be,maxdd
0,2017-06-22 15:00:00,2017-06-30 15:00:00,TA 1805,5030,5116,-1,7,-86,-3325.511,496674.489,-86.0,0.0,32.0
1,2017-07-03 15:00:00,2017-07-11 15:00:00,TA 1805,5116,5058,1,11,-58,-3685.957,492988.532,-58.0,0.0,66.0
2,2017-07-12 15:00:00,2017-08-07 15:00:00,TA 1805,5154,5222,1,9,68,2653.308,495641.840,-24.0,70.0,80.0
3,2017-08-09 15:00:00,2017-08-15 15:00:00,TA 1805,5340,5210,1,3,-130,-2085.825,493556.015,-130.0,0.0,36.0
4,2017-08-24 15:00:00,2017-08-29 15:00:00,TA 1805,5316,5216,1,7,-100,-3816.862,489739.153,-100.0,0.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2024-01-09 15:00:00,2024-01-18 15:00:00,TA 2405,5764,5850,-1,5,-86,-2379.035,626870.429,-86.0,0.0,62.0
127,2024-01-19 15:00:00,2024-01-31 15:00:00,TA 2405,5870,5962,1,9,92,3726.756,630597.185,0.0,0.0,84.0
128,2024-03-05 15:00:00,2024-03-15 15:00:00,TA 2405,5864,5928,-1,10,-64,-3658.960,626938.225,-64.0,0.0,64.0
129,2024-03-19 15:00:00,2024-03-22 15:00:00,TA 2405,5950,5806,1,7,-144,-5361.146,621577.079,-144.0,0.0,40.0


In [5]:
round(2.5001)

3